# Imports

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import sleep
import datetime as dt

In [2]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
from sqlalchemy import create_engine

In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\xengo\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


# Extract Data from Top 200

In [5]:
base = dt.date(2021, 9, 3)
end_list = [base - dt.timedelta(days=7*x) for x in range(5)]
end_list

[datetime.date(2021, 9, 3),
 datetime.date(2021, 8, 27),
 datetime.date(2021, 8, 20),
 datetime.date(2021, 8, 13),
 datetime.date(2021, 8, 6)]

In [6]:
base = dt.date(2021, 8, 27)
start_list = [base - dt.timedelta(days=7*x) for x in range(5)]
start_list 

[datetime.date(2021, 8, 27),
 datetime.date(2021, 8, 20),
 datetime.date(2021, 8, 13),
 datetime.date(2021, 8, 6),
 datetime.date(2021, 7, 30)]

In [7]:
urls = []
countriesList = ["global","us","th","in","br","mx","jp","ph","eg","vn","tr"]
for i in range(len(countriesList)):   
    for j in range(5):
        start_date = start_list[j].strftime("%Y-%m-%d")
        end_date = end_list[j].strftime("%Y-%m-%d")
        url = f"https://spotifycharts.com/regional/{countriesList[i]}/weekly/{start_date}--{end_date}"
        urls.append(url)
urls

['https://spotifycharts.com/regional/global/weekly/2021-08-27--2021-09-03',
 'https://spotifycharts.com/regional/global/weekly/2021-08-20--2021-08-27',
 'https://spotifycharts.com/regional/global/weekly/2021-08-13--2021-08-20',
 'https://spotifycharts.com/regional/global/weekly/2021-08-06--2021-08-13',
 'https://spotifycharts.com/regional/global/weekly/2021-07-30--2021-08-06',
 'https://spotifycharts.com/regional/us/weekly/2021-08-27--2021-09-03',
 'https://spotifycharts.com/regional/us/weekly/2021-08-20--2021-08-27',
 'https://spotifycharts.com/regional/us/weekly/2021-08-13--2021-08-20',
 'https://spotifycharts.com/regional/us/weekly/2021-08-06--2021-08-13',
 'https://spotifycharts.com/regional/us/weekly/2021-07-30--2021-08-06',
 'https://spotifycharts.com/regional/th/weekly/2021-08-27--2021-09-03',
 'https://spotifycharts.com/regional/th/weekly/2021-08-20--2021-08-27',
 'https://spotifycharts.com/regional/th/weekly/2021-08-13--2021-08-20',
 'https://spotifycharts.com/regional/th/week

In [8]:
artists = []
titles = []
streamCounts = []
dates = []
countryList = []
ranks = []
for u in urls:
    browser.visit(u)
    html = browser.html
    soup= BeautifulSoup(html, "lxml")
    songs= soup.find("table", {"class":"chart-table"})
    start = u.find("regional/") + len("regional/")
    end = u.find("/weekly")
    country = u[start:end]
    
    for tr in songs.find("tbody").find_all("tr"):
        rank = tr.find("td",{"class":"chart-table-position"}).text
        artist = tr.find("td", {"class": "chart-table-track"}).find("span").text
        artist= artist.replace("by ","").strip()
        title= tr.find("td",{"class": "chart-table-track"}).find("strong").text
        streamCount= tr.find("td", {"class": "chart-table-streams"}).text
        dateTemp = u.split("weekly/")[1]
        date = dateTemp.split("--")[1]
        ranks.append(rank)
        artists.append(artist)
        titles.append(title)
        streamCounts.append(streamCount)
        dates.append(date)
        countryList.append(country)
        
        

browser.quit()

In [9]:
#streamCountsNew = streamCounts[int(string.replace(',', '')) for x in range(len(streamCounts))]
for x in range(len(streamCounts)):
    temp = int(streamCounts[x].replace(',',''))
    streamCounts[x] = temp
    
dict_df = {
    "rank":ranks,
    "title":titles,
    "artist":artists,
    "streams":streamCounts,
    "country":countryList,
    "date":dates  
}

In [10]:
global_df = pd.DataFrame(dict_df)

In [11]:
global_df

,rank,title,artist,streams,country,date
0,1,STAY (with Justin Bieber),The Kid LAROI,68764542,global,2021-09-03
1,2,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,43663527,global,2021-09-03
2,3,Bad Habits,Ed Sheeran,34182580,global,2021-09-03
3,4,Beggin',Måneskin,32620597,global,2021-09-03
4,5,Hurricane,Kanye West,30609671,global,2021-09-03
...,...,...,...,...,...,...
10995,196,Küçücüğüm,Yalın,203594,tr,2021-08-06
10996,197,Siyah,"Patron, Sagopa Kajmer",201982,tr,2021-08-06
10997,198,Seni Buldum Ya,Kaan Boşnak,201945,tr,2021-08-06
10998,199,STAY (with Justin Bieber),The Kid LAROI,201409,tr,2021-08-06


# Extract Data from Viral 50

In [12]:
base_viral = dt.date(2021, 9, 2)
viral_dates = [base_viral - dt.timedelta(days=7*x) for x in range(5)]
viral_dates

[datetime.date(2021, 9, 2),
 datetime.date(2021, 8, 26),
 datetime.date(2021, 8, 19),
 datetime.date(2021, 8, 12),
 datetime.date(2021, 8, 5)]

In [13]:
#Viral 50
urlsViral = []
countriesList = ["global","us","th","in","br","mx","jp","ph","eg","vn","tr"]
for i in range(len(countriesList)):   
    for j in range(5):
        viral_date = viral_dates[j].strftime("%Y-%m-%d")
        #end_date = end_list[j].strftime("%Y-%m-%d")
        url = f"https://spotifycharts.com/viral/{countriesList[i]}/weekly/{viral_date}--{viral_date}"
        urlsViral.append(url)
len(urlsViral)

55

In [14]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\xengo\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [15]:
artists = []
titles = []
dates = []
countryList = []
ranks = []
for u in urlsViral:
    browser.visit(u)
    html = browser.html
    soup= BeautifulSoup(html, "lxml")
    songs= soup.find("table", {"class":"chart-table"})
    start = u.find("viral/") + len("viral/")
    end = u.find("/weekly")
    country = u[start:end]
    
    for tr in songs.find("tbody").find_all("tr"):
        rank = tr.find("td",{"class":"chart-table-position"}).text
        artist = tr.find("td", {"class": "chart-table-track"}).find("span").text
        artist= artist.replace("by ","").strip()
        title= tr.find("td",{"class": "chart-table-track"}).find("strong").text
        dateTemp = u.split("weekly/")[1]
        date = dateTemp.split("--")[1]
        ranks.append(rank)
        artists.append(artist)
        titles.append(title)
        dates.append(date)
        countryList.append(country)
browser.quit()

In [16]:

viral_dict = {
    "rank":ranks,
    "title":titles,
    "artist":artists,
    "country":countryList,
    "date":dates  
}

In [17]:
viral_df = pd.DataFrame(viral_dict)
viral_df

,rank,title,artist,country,date
0,1,It's Only Me - Studio Version,Kaleb J,global,2021-09-02
1,2,Chega E Senta - Ao Vivo,John Amplificado,global,2021-09-02
2,3,Manike Mage Hithe,"Chamath Sangeeth, Yohani, Satheeshan",global,2021-09-02
3,4,Now I Know,Kaleb J,global,2021-09-02
4,5,Passinho Debochado,Dan Ventura,global,2021-09-02
...,...,...,...,...,...
2745,46,Ölebilirim,"Bedo, Patron",tr,2021-08-05
2746,47,Bingo,Organize,tr,2021-08-05
2747,48,Anıları Sakla,Batuhan Kordel,tr,2021-08-05
2748,49,Flashbacks (Slowed),CRASPORE,tr,2021-08-05


# Load and Transform Data

In [18]:
country_dict = {"country":countriesList}
country_df = pd.DataFrame(country_dict)

In [19]:
country_df

,country
0,global
1,us
2,th
3,in
4,br
5,mx
6,jp
7,ph
8,eg
9,vn


In [20]:
viral_artist_unique = viral_df.artist.unique()
viral_artist_dict = {"artist":viral_artist_unique}
viral_artist_df = pd.DataFrame(viral_artist_dict)
viral_artist_df

,artist
0,Kaleb J
1,John Amplificado
2,"Chamath Sangeeth, Yohani, Satheeshan"
3,Dan Ventura
4,A-Wall
...,...
705,Lvbel C5
706,"Bedo, Patron"
707,Organize
708,CRASPORE


In [21]:
artist_unique = global_df.artist.unique()
artist_dict = {"artist":artist_unique}
artist_df = pd.DataFrame(artist_dict)
artist_df

,artist
0,The Kid LAROI
1,Lil Nas X
2,Ed Sheeran
3,Måneskin
4,Kanye West
...,...
1328,Stabil
1329,Hidra
1330,Irmak Arıcı
1331,"Patron, Sagopa Kajmer"


In [22]:
# Transform Data
artist_temp_df = pd.concat([artist_df,viral_artist_df]).drop_duplicates().reset_index(drop=True)


In [23]:
rds_connection_string = "postgres:booth@35.223.88.224:5432/Spotify"
engine = create_engine(f'postgresql://{rds_connection_string}') 

In [24]:
# Load Data
artist_temp_df.to_sql(name='artist', con=engine, if_exists='append', index=False) 


In [25]:
# Extract Data
new_artist_df = pd.read_sql_query('select * from artist', con=engine) 

In [26]:
# Load Data
country_df.to_sql(name='country', con=engine, if_exists='append', index=False)

In [27]:
# Extract Data
new_country_df = pd.read_sql_query('select * from country', con=engine) 
new_country_df

,country_id,country,last_updated
0,1,global,2021-09-14 01:38:45.297945
1,2,us,2021-09-14 01:38:45.297945
2,3,th,2021-09-14 01:38:45.297945
3,4,in,2021-09-14 01:38:45.297945
4,5,br,2021-09-14 01:38:45.297945
5,6,mx,2021-09-14 01:38:45.297945
6,7,jp,2021-09-14 01:38:45.297945
7,8,ph,2021-09-14 01:38:45.297945
8,9,eg,2021-09-14 01:38:45.297945
9,10,vn,2021-09-14 01:38:45.297945


In [28]:
# Transform Data
sub1 = global_df.merge(new_country_df,how="left",on="country")
sub2 = sub1.drop("country",1)
sub2

,rank,title,artist,streams,date,country_id,last_updated
0,1,STAY (with Justin Bieber),The Kid LAROI,68764542,2021-09-03,1,2021-09-14 01:38:45.297945
1,1,STAY (with Justin Bieber),The Kid LAROI,68764542,2021-09-03,12,2021-09-14 01:56:28.124757
2,1,STAY (with Justin Bieber),The Kid LAROI,68764542,2021-09-03,23,2021-09-14 02:03:53.394525
3,2,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,43663527,2021-09-03,1,2021-09-14 01:38:45.297945
4,2,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,43663527,2021-09-03,12,2021-09-14 01:56:28.124757
...,...,...,...,...,...,...,...
32995,199,STAY (with Justin Bieber),The Kid LAROI,201409,2021-08-06,22,2021-09-14 01:56:28.124757
32996,199,STAY (with Justin Bieber),The Kid LAROI,201409,2021-08-06,33,2021-09-14 02:03:53.394525
32997,200,Glock 2.0,"cakal, Reckol, Furkan Karakılıç, Yung Ouzo, TE...",201212,2021-08-06,11,2021-09-14 01:38:45.297945
32998,200,Glock 2.0,"cakal, Reckol, Furkan Karakılıç, Yung Ouzo, TE...",201212,2021-08-06,22,2021-09-14 01:56:28.124757


In [29]:
# Transform Data
sub3 = sub2.merge(new_artist_df,how="left",on="artist")
sub4 = sub3.drop("artist",1)
sub5 = sub4.drop("last_updated_y",1)
sub6 =sub5.rename(columns={"last_updated_x":"last_updated"})
final_global_df = sub6
final_global_df

,rank,title,streams,date,country_id,last_updated,artist_id
0,1,STAY (with Justin Bieber),68764542,2021-09-03,1,2021-09-14 01:38:45.297945,1
1,1,STAY (with Justin Bieber),68764542,2021-09-03,1,2021-09-14 01:38:45.297945,1763
2,1,STAY (with Justin Bieber),68764542,2021-09-03,1,2021-09-14 01:38:45.297945,3525
3,1,STAY (with Justin Bieber),68764542,2021-09-03,12,2021-09-14 01:56:28.124757,1
4,1,STAY (with Justin Bieber),68764542,2021-09-03,12,2021-09-14 01:56:28.124757,1763
...,...,...,...,...,...,...,...
98995,200,Glock 2.0,201212,2021-08-06,22,2021-09-14 01:56:28.124757,3095
98996,200,Glock 2.0,201212,2021-08-06,22,2021-09-14 01:56:28.124757,4857
98997,200,Glock 2.0,201212,2021-08-06,33,2021-09-14 02:03:53.394525,1333
98998,200,Glock 2.0,201212,2021-08-06,33,2021-09-14 02:03:53.394525,3095


In [30]:
final_global_df.to_sql(name='Top_200', con=engine, if_exists='append', index=False) 


In [31]:
# Transform Data
pd.read_sql_query("""select tt.title,a.artist, c.country 
from 
    "Top_200" tt
Join 
    "country" c on
        tt.country_id = c.country_id
join artist a on
    tt.artist_id = a.artist_id;
""", con=engine) 

,title,artist,country
0,STAY (with Justin Bieber),The Kid LAROI,global
1,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,global
2,Bad Habits,Ed Sheeran,global
3,Beggin',Måneskin,global
4,Hurricane,Kanye West,global
...,...,...,...
153995,Glock 2.0,"cakal, Reckol, Furkan Karakılıç, Yung Ouzo, TE...",tr
153996,Glock 2.0,"cakal, Reckol, Furkan Karakılıç, Yung Ouzo, TE...",tr
153997,Glock 2.0,"cakal, Reckol, Furkan Karakılıç, Yung Ouzo, TE...",tr
153998,Glock 2.0,"cakal, Reckol, Furkan Karakılıç, Yung Ouzo, TE...",tr


In [32]:
# Transform Data
sub1 = viral_df.merge(new_country_df,how="left",on="country")
sub2 = sub1.drop("country",1)
sub2

,rank,title,artist,date,country_id,last_updated
0,1,It's Only Me - Studio Version,Kaleb J,2021-09-02,1,2021-09-14 01:38:45.297945
1,1,It's Only Me - Studio Version,Kaleb J,2021-09-02,12,2021-09-14 01:56:28.124757
2,1,It's Only Me - Studio Version,Kaleb J,2021-09-02,23,2021-09-14 02:03:53.394525
3,2,Chega E Senta - Ao Vivo,John Amplificado,2021-09-02,1,2021-09-14 01:38:45.297945
4,2,Chega E Senta - Ao Vivo,John Amplificado,2021-09-02,12,2021-09-14 01:56:28.124757
...,...,...,...,...,...,...
8245,49,Flashbacks (Slowed),CRASPORE,2021-08-05,22,2021-09-14 01:56:28.124757
8246,49,Flashbacks (Slowed),CRASPORE,2021-08-05,33,2021-09-14 02:03:53.394525
8247,50,Ara Beni Cano,"Uro58, Grow",2021-08-05,11,2021-09-14 01:38:45.297945
8248,50,Ara Beni Cano,"Uro58, Grow",2021-08-05,22,2021-09-14 01:56:28.124757


In [33]:
# Transform Data
sub3 = sub2.merge(new_artist_df,how="left",on="artist")
sub4 = sub3.drop("artist",1)
sub5 = sub4.drop("last_updated_y",1)
sub6 =sub5.rename(columns={"last_updated_x":"last_updated"})
final_viral_df = sub6
final_viral_df

,rank,title,date,country_id,last_updated,artist_id
0,1,It's Only Me - Studio Version,2021-09-02,1,2021-09-14 01:38:45.297945,1334
1,1,It's Only Me - Studio Version,2021-09-02,1,2021-09-14 01:38:45.297945,3096
2,1,It's Only Me - Studio Version,2021-09-02,1,2021-09-14 01:38:45.297945,4858
3,1,It's Only Me - Studio Version,2021-09-02,12,2021-09-14 01:56:28.124757,1334
4,1,It's Only Me - Studio Version,2021-09-02,12,2021-09-14 01:56:28.124757,3096
...,...,...,...,...,...,...
24745,50,Ara Beni Cano,2021-08-05,22,2021-09-14 01:56:28.124757,3524
24746,50,Ara Beni Cano,2021-08-05,22,2021-09-14 01:56:28.124757,5286
24747,50,Ara Beni Cano,2021-08-05,33,2021-09-14 02:03:53.394525,1762
24748,50,Ara Beni Cano,2021-08-05,33,2021-09-14 02:03:53.394525,3524


In [34]:
# Load Data
final_viral_df.to_sql(name='Viral_50', con=engine, if_exists='append', index=False) 


In [35]:
pd.read_sql_query("""select vf.title,a.artist, c.country 
from 
    "Viral_50" vf
Join 
    "country" c on
        vf.country_id = c.country_id
join artist a on
    vf.artist_id = a.artist_id;
""", con=engine) 

,title,artist,country
0,It's Only Me - Studio Version,Kaleb J,global
1,Chega E Senta - Ao Vivo,John Amplificado,global
2,Manike Mage Hithe,"Chamath Sangeeth, Yohani, Satheeshan",global
3,Now I Know,Kaleb J,global
4,Passinho Debochado,Dan Ventura,global
...,...,...,...
38495,Ara Beni Cano,"Uro58, Grow",tr
38496,Ara Beni Cano,"Uro58, Grow",tr
38497,Ara Beni Cano,"Uro58, Grow",tr
38498,Ara Beni Cano,"Uro58, Grow",tr
